### Q1 如果有些表格填的是无 那么有内容时的内容格式一定是string吗
example: 审批结果的那些表头

### Q2 如果现在是汉字 后面是否一定是汉字
e.g. 匹配涉及到 [\u4e00-\u9fa9]

In [1]:
from selenium import webdriver
import re

import time

from bs4 import BeautifulSoup

from util import *
from RightPage import Right

import json

In [2]:
import os

In [3]:
from GuidePage import Guide

In [ ]:
# def find_parts(text):
#     '''
#     处理home page 得到所有部门的页面list
#     '''
#     part_dict = {}
#     pat = r'<li id="10400000...".*?><a href=".*?" onclick="queryByDept(.*?),this,.*?".*?>(.*?)</a></li>'
#     parts = re.findall(pat,text)
#     for a,b in parts:
#         a = a.replace('(','').replace("'",'')
#         b = b.replace(')','')
#         part_dict[b] = a
    
#     return part_dict

In [ ]:
# #测试RightPage 类可否解析
# turl = 'http://www.gdzwfw.gov.cn/portal/affairs-public-detail?qzqdCode=00288efa-bc84-4176-993f-8e08da4e56f0&deptCode=MB2C94128'
# browser = webdriver.Firefox(log_path = "webdriver.log")
# browser.get(turl)
# time.sleep(0.5)

# right = Right(turl,0)
# right.handle_page(browser)

# browser.close()

# for k,v in right.page.items():
#         print(k,v)
            
    

In [ ]:
# # 测试GuidePage 类可否解析
# # turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300MB2C94128A3440217281000'
# turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300693966093K3440101003001'
# # turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300693966093K3442001033001'
# browser = webdriver.Firefox(log_path = "webdriver.log")
# browser.get(turl)
# time.sleep(2)
# text = browser.page_source
# # browser.close()


# guide = Guide(browser,text,turl,0,0)
# guide.handle_guide()
# with open('3.json','w+',encoding='utf-8') as fout:
#     guide.disp(fout)
#     fout.write('***********分隔符******************')

In [4]:
path1 = 'sfdrive\geckodriver.exe'

In [5]:
# browser = webdriver.Firefox(executable_path = 'sfdrive\geckodriver.exe')

In [6]:
region_code = '''440000 广东省
440100 广州市
440400 珠海市
440500 汕头市 
440600 佛山市
440200 韶关市
441600 河源市 
441400 梅州市
441300 惠州市
441500 汕尾市
441900 东莞市
442000 中山市
440700 江门市
441700 阳江市
440800 湛江市
440900 茂名市
441200 肇庆市
441800 清远市
445100 潮州市
445200 揭阳市
445300 云浮市'''

In [9]:
# hurl = 'http://www.gdzwfw.gov.cn/portal/affairs-public-duty-list?region=440300&deptCode='
# hurl = 'http://www.gdzwfw.gov.cn/portal/affairs-public-duty-list?region=440300&deptCode=693966093'
# browser = webdriver.Firefox(log_path = "webdriver.log")
# browser = webdriver.Firefox(executable='fdriver/geckodriver.exe')
def run_spider(hurl,datapath):
    browser = webdriver.Firefox(executable_path = path1)
    browser.get(hurl)
    time.sleep(0.5)

    browser_rt = webdriver.Firefox(executable_path = path1)
    browser_gd = webdriver.Firefox(executable_path = path1)

    prefix = 'http://www.gdzwfw.gov.cn'

    text = browser.page_source 

    #### 关于 args 等信息 在py文件中修改

    count = 0
    count1 = 0
    rt_idx = 0
    while True:
        level1_url_list = find_urls(text)  ## 拿到主页下的list
  
        for lev_url in level1_url_list:
            lev_url = lev_url.replace('amp;','')
            url = prefix + lev_url
            browser_rt.get(url)
            time.sleep(1)

            right = Right(url,rt_idx)
            right.handle_page(browser_rt)
            
#             if '441900' in hurl:  ## 河源
#                 if rt_idx <= 3430:#130
#                     rt_idx += len(level1_url_list)
#                     flag = nextPage(browser,text)
#                     time.sleep(2)
#                     text = browser.page_source
#                     continue
            
            with open(datapath+str(rt_idx)+'.json','w+',encoding='utf-8') as fout:
                tmp = json.dumps(dict(right.page.items()),ensure_ascii=False,indent = 4)
                fout.write(tmp+'\n')
    #             fout.write('----------NextLevel----------')

                rt_idx += 1
   
                check_list = right.page['checklist_urls']

                pid = 0
                for curl in check_list:
                    pid += 1
                    ppid = str(rt_idx-1)+'_'+str(pid-1)
                    curl = prefix+curl

                    n = browser_gd.window_handles
                    browser_gd.switch_to_window(n[0])
                    browser_gd.get(curl)

                    count1 += 1
                    time.sleep(1)
                    text2 = browser_gd.page_source
                    try:
                        guide = Guide(browser_gd,text2,curl,rt_idx-1,ppid)
                        guide.handle_guide()
                        guide.disp(fout)
                    except:
                        pass

                    if len(guide.qas.content)!=0:
                        browser_gd.close()


    #     break
        flag = nextPage(browser,text)
    #     print(flag)
        if flag == 0:
            break

        time.sleep(2)
        text = browser.page_source
        count += 1
    #     if count >= 2:
    #         break

    browser.close()
    browser_rt.close()
    browser_gd.close()

In [8]:
region = region_code.split('\n')
# print(len(region))

In [ ]:
# swh 01
# cfx 23
# zyy 45
# xc 67
# lzx 89
# wzz 10 11
# cyg 12 13 师姐 fny
# lcn 14 15
# wxp 16 17
# cy 18 19
# wg 20
# sz = ['440300 深圳市']

# hurl = 'http://www.gdzwfw.gov.cn/portal/affairs-public-duty-list?region=440300&deptCode=693966093'
# run_spider(hurl,'data1/')

for reg in region[4:5]:
# for reg in sz:
    reg = reg.split(' ')
    datapath = reg[1]+'/'
    if not os.path.exists(datapath):
        os.makedirs(datapath)
    hurl = 'http://www.gdzwfw.gov.cn/portal/affairs-public-duty-list?region={}&deptCode='.format(reg[0])
    run_spider(hurl,datapath)